In [14]:
import os
import pandas as pd
import uuid

In [5]:
df = pd.read_json(
    f'/home/admin123/AWS-Lambda/notebook/data/2022-06-05-0.json.gz',
    lines=True,
    orient='records'
)

In [11]:
file_name='2022-06-05-0.json.gz'

In [18]:
# Read the data from json file in chunks
# Reader type object will be created rather than Dataframe

df_reader = pd.read_json(
    f'/home/admin123/AWS-Lambda/notebook/data/2022-06-05-0.json.gz',
    lines=True,
    orient='records',
    chunksize=10000
)

In [ ]:
# We can iterate through reader in streaming fashion using for loop

for idx, df in enumerate(df_reader):
    print(f' The number of recors in chunk{idx}:{df.shape[0]}')
    

In [12]:
year = file_name.split('-')[0]
month = file_name.split('-')[1]
dayofmonth = file_name.split('-')[2]
hour = file_name.split('-')[3].split('.')[0]

In [15]:
target_file_name = f'part-{year}-{month}-{dayofmonth}-{hour}-{uuid.uuid1()}.snappy.parquet'
target_file_name

'part-2022-06-05-0-07fff496-4d67-11ee-8907-00155d2c3f2b.snappy.parquet'

In [19]:
for idx, df in enumerate(df_reader):
    target_file_name = f'part-{year}-{month}-{dayofmonth}-{hour}-{uuid.uuid1()}.snappy.parquet'
    print(f'Processing chunk {idx} of size {df.shape[0]} from {file_name}')
    df.drop(columns=['payload']). \
    to_parquet(
        f's3://aigithubs/raw/ghactivity/year={year}/month={month}/dayofmonth={dayofmonth}/{target_file_name}',
        index=False
    )

Processing chunk 0 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 1 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 2 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 3 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 4 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 5 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 6 of size 10000 from 2022-06-05-0.json.gz


OSError: Write failed: ConnectionClosedError('Connection was closed before we received a valid response from endpoint URL: "https://aigithubs.s3.amazonaws.com/raw/ghactivity/year%3D2022/month%3D06/dayofmonth%3D05/part-2022-06-05-0-8e5c19d8-4d68-11ee-8907-00155d2c3f2b.snappy.parquet?partNumber=1&uploadId=fDDPyx8Tnstj5ebZlB1AO077jzueq618DI72h3wWgoRRtEuzKr7QBASJgK1SvUoOakOpKBRT4s7HIOBTRAIGFg--".')

In [ ]:
!aws s3 ls s3://aigithubs/raw/ghactivity/ --recursive